In [58]:
import torch
import pandas as pd
import numpy as np
import torchvision
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision.models import *
import torch.nn as nn
import time
import os
import optuna
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error, recall_score

In [34]:
class Muse_Dataset(Dataset):
    def __init__(self, datapath):
        super().__init__()
        self.df = pd.read_csv(datapath)
        self.labels = self.df['class_id']
        self.features = self.df.loc[:,'0':'87']
        
        self.features =torch.from_numpy(self.features.values)
        self.labels = torch.from_numpy(self.labels.values)
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, index):
        feature = self.features[index]
        label = self.labels[index]
        return feature, label

In [35]:
torch.manual_seed(0)

learning_rate = 0.1
batch_size = 256
num_epochs = 10
num_classes = 3
device = "cuda:0"
feature_set = 'egemaps'

In [36]:
train_dataset = Muse_Dataset('./data_csv/'+'arousal/'+feature_set+'_train.csv')

train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=batch_size,
                          num_workers=4,
                          shuffle=True)
print('./data_csv/'+feature_set+'_train.csv')

./data_csv/egemaps_train.csv


In [37]:
devel_dataset = Muse_Dataset('./data_csv/'+'arousal/'+feature_set+'_devel.csv')

devel_loader = DataLoader(dataset=devel_dataset, 
                          batch_size=batch_size,
                          num_workers=4,
                          shuffle=True)


In [50]:
class Wave_Block(nn.Module):

    def __init__(self, in_channels, out_channels, dilation_rates, kernel_size):
        super(Wave_Block, self).__init__()
        self.num_rates = dilation_rates
        self.convs = nn.ModuleList()
        self.filter_convs = nn.ModuleList()
        self.gate_convs = nn.ModuleList()

        self.convs.append(nn.Conv1d(in_channels, out_channels, kernel_size=1))
        dilation_rates = [2 ** i for i in range(dilation_rates)]
        for dilation_rate in dilation_rates:
            self.filter_convs.append(
                nn.Conv1d(out_channels, out_channels, kernel_size=kernel_size, padding=int((dilation_rate*(kernel_size-1))/2), dilation=dilation_rate))
            self.gate_convs.append(
                nn.Conv1d(out_channels, out_channels, kernel_size=kernel_size, padding=int((dilation_rate*(kernel_size-1))/2), dilation=dilation_rate))
            self.convs.append(nn.Conv1d(out_channels, out_channels, kernel_size=1))

    def forward(self, x):
        x = self.convs[0](x)
        res = x
        for i in range(self.num_rates):
            x = torch.tanh(self.filter_convs[i](x)) * torch.sigmoid(self.gate_convs[i](x))
            x = self.convs[i + 1](x)
            res = res + x
        return res
# detail 
class wavenet(nn.Module):
    def __init__(self, inch=1, kernel_size=3, num_classes =3):
        super().__init__()
        #self.LSTM = nn.GRU(input_size=input_size, hidden_size=64, num_layers=2, batch_first=True, bidirectional=True)
        self.wave_block1 = Wave_Block(inch, 16, 4, kernel_size)
        self.wave_block2 = Wave_Block(16, 32, 4, kernel_size)
        self.wave_block3 = Wave_Block(32, 64, 2, kernel_size)
        self.wave_block4 = Wave_Block(64, 128, 1, kernel_size)
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        print(x.shape)
        x = x.permute(0, 2, 1)

        x = self.wave_block1(x)
        x = self.wave_block2(x)
        x = self.wave_block3(x)

        x = self.wave_block4(x)
        x = x.permute(0, 2, 1)
        #x, _ = self.LSTM(x)
        x = self.fc(x)
        return x

In [51]:
model = wavenet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=1e-4)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [52]:

model.to(device)
print('done')

done


In [53]:
def f1(y_true, y_pred):
    return round(f1_score(y_true, y_pred, average='micro') * 100, 3)


def uar(y_true, y_pred):
    return round(recall_score(y_true, y_pred, average='macro') * 100, 3)


def eval_metric(predicts, targets, partition_name):
    results = {}
    results['f1'] = f1(targets, predicts)
    results['uar'] = uar(targets, predicts)
    results['combine'] = round((0.66 * results['f1'] + 0.34 * results['uar']), 3)
    print(f'Results in {partition_name}:\n')
    print("  - f1: ", results['f1'])
    print("  - uar: ", results['uar'])
    print("  - combined:", results['combine'])

In [55]:
start_time = time.time()

cost_list=[]
schedule = [
    (0, 50, .1),
    (50, 100, .01),
    (100, 150, .001)]

for start, end, lr in schedule:

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    for epoch in range(start, end):
        model.train()
        for batch_idx, (features, targets) in enumerate(train_loader):


            features = features.unsqueeze(-1).float()
            targets = targets.long()

            features = features.to(device)
            targets = targets.to(device)

            ### FORWARD AND BACK PROP
            out = model(features)
            
            loss = criterion(out, targets)
            optimizer.zero_grad()

            loss.backward()

            ### UPDATE MODEL PARAMETERS
            optimizer.step()

            #################################################
            ### CODE ONLY FOR LOGGING BEYOND THIS POINT
            ################################################
            cost_list.append(loss.item())
            if  not batch_idx % 20:
                print (f'Epoch: {epoch+1:03d}/{end:03d} | '
                       f'Batch {batch_idx:03d}/{len(train_loader):03d} |' 
                       f' Cost: {loss:.4f}')

        model.eval()
        with torch.no_grad(): 
            pred=[]
            targ=[]
            for features, targets in devel_loader:
                features = features.view(-1, 1, 32, 16).float() 
                features = features.to(device)

                out = model(features)
                _, predicts = torch.max(out, 1)
                predicts = predicts.cpu().detach().numpy()
                pred.append(predicts)
                targ.append(targets)
            targ = [item for sublist in targ for item in sublist]
            pred = [item for sublist in pred for item in sublist]
            eval_metric(pred, targ, 'arosal')

        elapsed = (time.time() - start_time)/60
        print(f'Time elapsed: {elapsed:.2f} min')

    elapsed = (time.time() - start_time)/60
    print(f'Total Training Time: {elapsed:.2f} min')

torch.Size([256, 88, 1])


ValueError: Expected target size (256, 3), got torch.Size([256])